In [ ]:
import sys
import os

os.chdir("..")
from modules.data.download import download_data
download_data("2025-06-01", "2025-07-01")

In [5]:
import sys
import os

os.chdir("..")

from modules.points.generate import generate_points
generate_points(batch_size=200, filters={"start_date": "2025-06-01", "end_date": "2025-07-01"})

Starting analysis pass 1 with batch size 200
Found 200 unanalysed debates. Processing...
Processing debate 1/200: Reoffending: Young Offenders (ID: 5A82A20B-C157-48C0-89C0-A85E2DCE678D)
Processing 12 contributions for debate 5A82A20B-C157-48C0-89C0-A85E2DCE678D
Processing contribution 1/12 in debate 5A82A20B-C157-48C0-89C0-A85E2DCE678D
Processing contribution 4/12 in debate 5A82A20B-C157-48C0-89C0-A85E2DCE678D
Skipping contribution 44054947 - no member ID
Processing contribution 7/12 in debate 5A82A20B-C157-48C0-89C0-A85E2DCE678D
Processing contribution 10/12 in debate 5A82A20B-C157-48C0-89C0-A85E2DCE678D
Debate 5A82A20B-C157-48C0-89C0-A85E2DCE678D marked as analysed.
Processing debate 2/200: USAID Funding Pause (ID: 5A8B70D7-5040-4736-AE07-772BFF0A8AF7)
Processing 65 contributions for debate 5A8B70D7-5040-4736-AE07-772BFF0A8AF7
Processing contribution 1/65 in debate 5A8B70D7-5040-4736-AE07-772BFF0A8AF7
Skipping contribution 3260217 - no member ID
Skipping contribution 44150509 - no me

BadRequestError: Error code: 400 - {'error': {'message': 'Organization has blocked API access because a spend alert threshold was met. Please visit https://console.groq.com/settings/billing to manage your spend alerts.', 'type': 'invalid_request_error', 'code': 'spend_limit_reached'}}

In [ ]:
import psycopg2
from typing import Dict
import base64, json

def get_db_connection():
    return psycopg2.connect(
        host="localhost",
        database="parliament",
        user="parliament_user",
        password="parliament_pass",
        port=5432
    )

def create_cluster_jobs_and_clusters_tables():
    conn = get_db_connection()
    cur = conn.cursor()
    # Create cluster_jobs table
    cur.execute("""
    CREATE TABLE IF NOT EXISTS cluster_jobs (
      job_id        BIGSERIAL PRIMARY KEY,
      status        TEXT NOT NULL CHECK (status IN ('queued','running','complete','failed','canceled')),
      root_cluster_id INTEGER REFERENCES clusters(cluster_id) ON DELETE SET NULL,
      params        JSONB NOT NULL,
      created_at    TIMESTAMPTZ NOT NULL DEFAULT now(),
      started_at    TIMESTAMPTZ,
      finished_at   TIMESTAMPTZ,
      error         TEXT
    );
    """)
    # Add job_id and is_draft columns to clusters table if not exist
    cur.execute("""
    ALTER TABLE clusters
    ADD COLUMN IF NOT EXISTS job_id BIGINT REFERENCES cluster_jobs(job_id);
    """)
    cur.execute("""
    ALTER TABLE clusters
    ADD COLUMN IF NOT EXISTS is_draft BOOLEAN NOT NULL DEFAULT FALSE;
    """)
    conn.commit()
    cur.close()
    conn.close()

create_cluster_jobs_and_clusters_tables()